## Workflow Steps
Read in your .csv files into Pandas DataFrame object.
Create InMemory SQLite powered Sqlalchemy engine.
Cast Pandas DataFrame object to SQL Engine.
Create an SQL Database object through Llama Index.
Use the SQL Database object to create a Query Engine Tool that we can interact through text-to-SQL capabilities (NLSQLTableQueryEngine).

In [ ]:
!pip install -qU llama_index sqlalchemy pandas
!pip install openai

In [2]:
from google.colab import userdata
import openai
import os
#
from sqlalchemy import create_engine
from llama_index import SQLDatabase
from llama_index.llms import OpenAI
from llama_index.embeddings import OpenAIEmbedding
from llama_index import ServiceContext
from llama_index.indices.struct_store.sql_query import NLSQLTableQueryEngine
from llama_index.tools.query_engine import QueryEngineTool
from llama_index.agent import OpenAIAgent

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
openai_api_key = userdata.get('OPENAI_API_KEY')

## Load CSV file to pandas DataFrame

In [ ]:
df = pd.read_csv("/content/BA_AirlineReviews.csv")
df.head()

In [ ]:
# Drop the unwanted columns
df = df.drop(columns=['Unnamed: 0'])
df.head()

In [ ]:
# Check the % of null values with each column attribute
df.isnull().sum()[df.isnull().sum() > 0] / len(df) * 100.0

In [ ]:
temp = df.dropna()

In [ ]:
engine = create_engine('sqlite://', echo=False)
temp.to_sql('reviews', con=engine)

## Construct a SQLDatabase Index

In [ ]:
reviews_db = SQLDatabase(engine,include_tables=['reviews'])

## Instantiate the LLM and ServieContext

In [ ]:
llm = OpenAI(model_name="gpt-4-0613",
             temperature=0,
             streaming=True)
embedding = OpenAIEmbedding()
#
servicecontext = ServiceContext.from_defaults(llm=llm,
                        embed_model=embedding,
                        chunk_size=500)

use `NLSQLTableQueryEngine` to construct natural language queries that are synthesized into SQL queries

In [ ]:
sql_query_engine = NLSQLTableQueryEngine(sql_database=reviews_db,
                      tables=['reviews'],
                      service_context=servicecontext)
description = ("Provides information about airlines reviews from reviews table. "
                "Use a detailed plain text question as input to the tool.")

### Wrap all the details into QueryEngineTool

In [ ]:
sql_tool = QueryEngineTool.from_defaults(query_engine=sql_query_engine,
                      name="sql_query",
                      description=description)
#
agent = OpenAIAgent.from_tools(tools=[sql_tool],verbose=True)

### Generate the response

In [ ]:
## Question 1
response = agent.chat("What are the top 10 bad ReviewBody?")

In [ ]:
## Question 2
sql_tool = QueryEngineTool.from_defaults(query_engine=sql_query_engine,
                     name="sql_query",
                     description=description)
#
agent = OpenAIAgent.from_tools(tools=[sql_tool],verbose=True)
#
response = agent.chat("What is the highest overall rating provided by travellers?")